In [1]:
import numpy as np
import random
import math
import time
import pandas as pd

In [2]:
def generate_deltaset(N,p,sigma2):
    mean = [0,0,0]
    cov = np.zeros([p,p])
    np.fill_diagonal(cov,sigma2)
    delta=np.random.multivariate_normal(mean,cov,N).T  ##维度是p*N，单独拿出来一列是个列表，必须强制转成矩阵才能方便后续操作
    
    std = np.sqrt(sigma2)
    delta_k = np.zeros([p,p])
    for i in range(p):
        delta_k[i,i] = np.random.normal(0,std)
    return [delta,delta_k]

In [3]:
## 该函数输出Sigma_1_hat, Sigma_2_hat以及由二者组成的渐近方差
## 兰总后来做了稍微调整的算法
def estimate_Sigma(p,N,X,Y,delta,delta_k,sigma2,betahat):
    loss = np.zeros([N,1])   
    loss_delta = np.zeros([N,1])
    Sigma_1_tmp_part1 = np.zeros([N,p,p])
    Sigma_2_tmp_part1 = np.zeros([N,p,p])
    for i in range(N):
        Xbeta = np.dot(X[i,:],betahat)  ## X * beta
        delta_tmp = np.array([delta[:,i]]).T  ## p*1
        Xbeta_delta = np.dot(X[i,:],(betahat+delta_tmp))  ## X *(beta+delta)
        loss[i] = -Y[i]*Xbeta + np.log(1+np.exp(Xbeta))  ## 逻辑回归的损失函数是似然函数的负数
        loss_delta[i] = -Y[i]*Xbeta_delta + np.log(1+np.exp(Xbeta_delta))
        dd = np.dot(delta_tmp,delta_tmp.T) ## delta * t(delta)  p*p    
        Sigma_1_tmp_part1[i,:,:] = dd*((loss_delta[i] - loss[i])**2)  ## p*p
        Sigma_2_tmp_part1[i,:,:] = dd*(loss_delta[i] - loss[i])
    Sigma_1_hat_part1 = np.sum(Sigma_1_tmp_part1,0)/2/N/sigma2/sigma2
    Sigma_2_hat_part1 = np.sum(Sigma_2_tmp_part1,0)/N/sigma2/sigma2
    
    Sigma_1_hat = Sigma_1_hat_part1 - np.eye(p)*np.trace(Sigma_1_hat_part1)/(p+2)
    Sigma_2_hat = Sigma_2_hat_part1 - np.eye(p)*np.trace(Sigma_2_hat_part1)/(p+2)
    
    cov = np.dot(np.dot(np.linalg.inv(Sigma_2_hat),Sigma_1_hat),np.linalg.inv(Sigma_2_hat))/N ## 三明治形式方差
    #cov = np.linalg.inv(Sigma_1_hat)/N     ## 只利用Sigma_1_hat求解方差
    #cov = np.linalg.inv(Sigma_2_hat)/N     ## 只利用Sigma_2_hat求解方差
    return[Sigma_1_hat,Sigma_2_hat,cov]

In [4]:
######### 小样本情况，RMSE，MLE的SE，Bootstrap的SE，Mestimator的SE
N = [500,1000,2000]
beta = np.array([[1],[0.5],[0.5]])
p = np.size(beta)
nsimu = 100
f = open("SimuResult_Case2_SmallSample_2.txt", 'w')
f.write('N &');f.write('M_se &');f.write('M_time \\\\ \n');

## 不同方法的估计量
M_se_result = np.zeros([len(N),p])

## 不同方法的时间
m_time = np.zeros([len(N),1])

for i in range(len(N)):
    #beta_tmp = np.zeros([nsimu,p])
    betadat = pd.read_csv('beta'+str(N[i])+'.csv')
    betahat_full = betadat.values
    #MLE_se_tmp = np.zeros([nsimu,p])
    cov_tmp = np.zeros([nsimu,p])

    m_time_tmp = np.zeros([nsimu,1])
    
    sigma2 = N[i]**(-0.25)
    np.random.seed(2020)
    for j in range(nsimu):
        print(N[i],j)
        [delta,delta_k] = generate_deltaset(N[i],p,sigma2)
        XY=pd.read_csv('./outputXY_3/XY'+str(N[i])+'_'+str(j)+'.csv')
        Xdat = XY[["X1","X2","X3"]]
        Ydat = XY[["Y"]]
        X = Xdat.values
        Y = Ydat.values
        betahat = np.array([betahat_full[j,:]]).T
        start2 = time.time();[Sigma_1_hat,Sigma_2_hat,cov] = estimate_Sigma(p,N[i],X,Y,delta,delta_k,sigma2,betahat);end2 = time.time() ## 我们的方法
        
        #beta_tmp[j,:] = np.transpose(betahat)
        cov_tmp[j,:] = np.array(cov.diagonal())
        #MLE_se_tmp[j,:] = np.transpose(sebeta)
        #beta_boot_tmp[j,:] = np.transpose(beta_boot)

        #mle_time_tmp[j] = end1 - start1
        m_time_tmp[j] = end2 - start2
        #boot_time_tmp[j] = end3 - start3
    M_se_tmp = np.sqrt(cov_tmp)
    #bias_beta = beta_tmp - np.transpose(beta)
    #beta_result[i,:] = np.sqrt(np.sum(np.multiply(bias_beta,bias_beta),0)/nsimu)  ## 真实的差距
    #beta_boot_result[i,:] = np.mean(beta_boot_tmp,0) ## bootstrap的SE
    #MLE_se_result[i,:] = np.mean(MLE_se_tmp,0)    ## MLE的SE
    M_se_result[i,:] = np.mean(M_se_tmp,0)        ## 我们的方法的SE
    
    #mle_time[i] = np.mean(mle_time_tmp)
    #boot_time[i] = np.mean(boot_time_tmp)
    m_time[i] = np.mean(m_time_tmp)
    
    #MLE_RMSE = np.round(beta_result[i,:],decimals = 4)
    #BOOT_SE = np.round(beta_boot_result[i,:],decimals = 4)
    #MLE_SE = np.round(MLE_se_result[i,:],decimals = 4)
    M_SE = np.round(M_se_result[i,:],decimals = 4)
    
    #MLE_time = np.round(mle_time[i],decimals = 3)
    #BOOT_time = np.round(boot_time[i],decimals = 3)
    M_time = np.round(m_time[i],decimals = 3)
    f.write(str(N[i])+'&');f.write(f'{M_SE} &');f.write(f'{M_time} \\\\ \n');
f.close()

500 0
500 1
500 2
500 3
500 4
500 5
500 6
500 7
500 8
500 9
500 10
500 11
500 12
500 13
500 14
500 15
500 16
500 17
500 18
500 19
500 20
500 21
500 22
500 23
500 24
500 25
500 26
500 27
500 28
500 29
500 30
500 31
500 32
500 33
500 34
500 35
500 36
500 37
500 38
500 39
500 40
500 41
500 42
500 43
500 44
500 45
500 46
500 47
500 48
500 49
500 50
500 51
500 52
500 53
500 54
500 55
500 56
500 57
500 58
500 59
500 60
500 61
500 62
500 63
500 64
500 65
500 66
500 67
500 68
500 69
500 70
500 71
500 72
500 73
500 74
500 75
500 76
500 77
500 78
500 79
500 80
500 81
500 82
500 83
500 84
500 85
500 86
500 87
500 88
500 89
500 90
500 91
500 92
500 93
500 94
500 95
500 96
500 97
500 98
500 99
1000 0
1000 1
1000 2
1000 3


C:\Users\Jing\anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in sqrt


1000 4
1000 5
1000 6
1000 7
1000 8
1000 9
1000 10
1000 11
1000 12
1000 13
1000 14
1000 15
1000 16
1000 17
1000 18
1000 19
1000 20
1000 21
1000 22
1000 23
1000 24
1000 25
1000 26
1000 27
1000 28
1000 29
1000 30
1000 31
1000 32
1000 33
1000 34
1000 35
1000 36
1000 37
1000 38
1000 39
1000 40
1000 41
1000 42
1000 43
1000 44
1000 45
1000 46
1000 47
1000 48
1000 49
1000 50
1000 51
1000 52
1000 53
1000 54
1000 55
1000 56
1000 57
1000 58
1000 59
1000 60
1000 61
1000 62
1000 63
1000 64
1000 65
1000 66
1000 67
1000 68
1000 69
1000 70
1000 71
1000 72
1000 73
1000 74
1000 75
1000 76
1000 77
1000 78
1000 79
1000 80
1000 81
1000 82
1000 83
1000 84
1000 85
1000 86
1000 87
1000 88
1000 89
1000 90
1000 91
1000 92
1000 93
1000 94
1000 95
1000 96
1000 97
1000 98
1000 99
2000 0
2000 1
2000 2
2000 3
2000 4
2000 5
2000 6
2000 7
2000 8
2000 9
2000 10
2000 11
2000 12
2000 13
2000 14
2000 15
2000 16
2000 17
2000 18
2000 19
2000 20
2000 21
2000 22
2000 23
2000 24
2000 25
2000 26
2000 27
2000 28
2000 29
2000 30
